# Read Section 8 of the Suplementary Information 
# for details software utilization

In [ ]:
# common library
import numpy as np
import math 
import cmath 
from pylab import *
import matplotlib.pyplot as plt
import scipy.special
from scipy import optimize
from scipy.optimize import curve_fit
from scipy import integrate
from scipy.interpolate import UnivariateSpline
from scipy.optimize import minimize, rosen, rosen_der, differential_evolution
import copy




# Vector rotation functions
#############################################

def rot_zaxis(vec, theta):     #ve_ax=rot_zaxis(r,ang)#r_matr[2]
    "Return vector rotated around Z axis by theta radians"
    rotmat = np.array(([np.cos(theta),-np.sin(theta),0],[np.sin(theta), np.cos(theta),0],[0,0,1]))
    rotvec=np.inner(rotmat,vec)
    return rotvec

def rot_xaxis(vec, theta):
    "Return vector rotated around X axis by theta radians"
    rotmat = np.array((   [1,0,0],[0 , np.cos(theta), -np.sin(theta)] , [0,np.sin(theta), np.cos(theta)]))
    rotvec=np.inner(rotmat,vec)
    return rotvec

def rot_yaxis(vec, theta):
    "Return vector rotated around X axis by theta radians"
    rotmat = np.array(([ np.cos(theta),0, np.sin(theta)] ,[0,1,0], [-np.sin(theta),0, np.cos(theta)]))
    rotvec=np.inner(rotmat,vec)
    return rotvec

def mi_zaxis(vec):
    "Return vector rotated around X axis by theta radians"
    mimat = np.array(([ 0,0, 1] ,[0,1,0], [-1,0,0]))
    rotvec=np.inner(mimat,vec)
    return rotvec

### Genral function

# Finds the direct beam 
def i_dire(h, k,l):
    for i in range(len(k)):
        if (h[i] == 0 and  k[i] == 0 and l[i]==0):
            return i
        
### Residue Calculation
### See Equation 7
def Residue(B,pre,tt,mdb,y_exp,i):
    
    y_med = copy.deepcopy(y_exp) 
    y_med = np.delete(y_med,i) # excludes the direct beam
    y_med = np.sqrt(np.array(y_med))
    
    y_sim = (I_Func(B[0],B[1],B[2],pre,tt,mdb)) 
    y_sim = np.delete(y_sim,i)
    y_sim = np.sqrt(np.array(y_sim))

    dif = abs(y_med - y_sim)
    return np.mean(dif)/np.mean(y_med)

In [ ]:
# Atomic Scattering Factor 

##Kirkland parametrization F(q)  q= sin(theta)/Lambda   

# calculation atomic scattering factor for In and P

def at_fq(q,param):
    feq=0
    for n1 in range(0,3):
        feq=feq+param[2*n1]/(q*q+param[2*n1+1])+param[2*n1+6]*np.exp(-param[2*n1+7]*q*q)
    return feq

############################
# User Provided
##############################
#Kirkland, E. J. (2020). Advanced Computing in Electron Microscopy. Cham, Switzerland: Springer.
# Table of Kirkland parametrization is organized in:
# a1,b1   a2,b2
# a3,b3   c1,d1
# c2,d2   c3,d3

Par= [a1, b1, a2, b2,\
a3, b3, c1, d1,\
c2, d2, c3, d3] 
####################################
q_co=[]
for n1 in range(0,51):
    q_co.append(2* n1/50)

funk = []
for n1 in range(0,len(q_co)):
    funk.append(at_fq(q_co[n1],Par))
    
f_q=UnivariateSpline(q_co, funk, k=3,s=0.00001 ) # interpolationto obtain a continuos curve with controled amount of points

# Structure Factor 

In [ ]:
#############################################

# Scattering angle 
def scatangl(h1,k1,l1,lam1,a0,b0):
    ### User input: crystal primitive dimensions and its projection on the zone axis  
    d_inv =  ??? #defined reciprocal d_spacing for the user crystal zone axis  
    scang=np.arctan(lam  *  np.sqrt(d_inv))
    return scang

# Struture Factor
def str_fac_f2(h1,k1,l1,lam1,a0,b0,mul): 

    ### User input: crystal primitive dimensions and its projection on the zone axis  
    
    san= scatangl(k1,h1,l1,lam1,a0,b0) # scattering vector norm
    f= f_q(np.sin(san)/lam1/10) #atomic scatting factor 
    
    #############################################

    DB_fac = ??? # Debye-Waller factor for bulk and room temperature (angstron squared)

    db1 = np.exp(-mul*DB_fac*((np.sin(san)/lam1/10)**2))
    # Structure factor for the studied crytal system
    F_sum = ???  'Equation for the crystal structure factor'
    F_f2= norm(F_sum)
    return F_f2

###################################################################


# Excitaton error
## See Equation 4 in paper

In [ ]:
##################################################################
def excerr_precang(the1,phi1,g1,lam1):     #ve_ax=rot_zaxis(r,ang)#r_matr[2]
    "Return excit error as functio of azimangle, prec ang, recip vector, K, "
    sg1=-np.sin(phi1)*norm([g1[0],g1[1]])*np.cos(the1)-g[2]-norm(g)**2*lam1/2
    return sg1

##############################################################################


# Two-beam Intensity 
## See Equation 2 in paper

In [ ]:
def Int_TB_exc_er(the2,phi2,g2,lam2,a2,b2,c2,f2):     #ve_ax=rot_zaxis(r,ang)#r_matr[2]
    "Return two beam intensity for vectorg resprsnted by its excit error and Fhkl"
    s2= -np.sin(phi2)*norm([g2[0],g2[1]])*np.cos(the2)-g2[2]-norm(g2)**2*lam2/2
    twobint = c2   *  ((a2*f2/b2)* np.sinc( a2* np.sqrt(s2**2+(f2/b2)**2) ))**2
    return twobint

# Integrated Intensity 
## See Equation 3 in paper

In [ ]:
#Função que calcula a intensidade 

##############################################################################################
def I_Func(ang_plane,ang_z,cte,pre,espe,mul):
    
    # user defined primitive lenght (2D)
    a0 = ???
    b0 = ???
    # user defined versor for the diffraction pattern zone axis 
    a = ???  # X versor of the 2D diffraction pattern
    b = ???  # Y versor of the 2D diffraction pattern
    
    prim_vol = ???  # primitive cell volume
    
    prec_ang =  ((pre/180)*np.pi) + al_diverg  # angle of precession
    ang_par = ang_plane/180*np.pi  # angle of the vector parallel to the zone axis
    ang_per = ang_z/180* np.pi     # angle of the vector perpendicular to the zone axis 

    integr_sg_n=[]
    for n1 in range(0,len(h_n)):
        aop0= espe  #nm thickness
        # unit cell volume provided by user
        bop0= np.pi *(prim_vol)/lam  # unit-cel col pi/lam
        cop0= abs(cte) #scaling
        h=h_n[n1]
        k=k_n[n1]
        l=l_n[n1]

    # reciprocal space vector
    # user must provide a rotation system related to the zone axis of the measurement (see S.3 in Supplementary Information)
    # the g vector in the reference system must be the linear combination of versor and hkl
        geve = ???
    ############# integration on az angle theta 0-2 Pi due to PED 

        F=str_fac_f2(h,k,l,lam,a0,b0,mul)
        a_rot = 10/180*np.pi # integration dummy variable
        y=integrate.quad(Int_TB_exc_er, a_rot,(a_rot+2.*np.pi), args=(prec_ang,geve,lam,aop0,bop0,cop0,F,))
        integr_sg_n.append(y[0])    #integrated by the rotating ewald spheres

    return integr_sg_n
##############################################################################################

# Orientation Determination

In [ ]:
# User definided

# Debye-Waller and thickness grid
# choose value with care as the optimization time is greatly influenced by it

mf_init = ???     # initial value for the mdb factor (see Eq. 6)
mf_endt = ???     # final value for the mdb factor (see Eq. 6)
mf_step = ???     # step value for the mdb factor (see Eq. 6)
t_init = ???      # initial value for the thickness [nm] (see Eq. 6)
t_endt = ???      # final value for the thickness [nm](see Eq. 6)
t_step = ???      # step value for the thickness [nm] (see Eq. 6)

# some experimental paramenters
pre = ???       # precession angle[deg]
al_diverg = ???  # divergence angle [deg]
lam = ???         # electron beam wavelenght [nm] 

# fit quantities 

beta0 = ???       # initial value of the plane 
phi0 = ???        # inital value of the c-axis orientation
N0 = ???          # inital scale factor
init_val = [beta0, phi0, N0] 

# limits of the fit if necessary
beta_init = ???     # initial value for the beta factor (see Eq. 6)
beta_endt = ???     # final value for the beta factor (see Eq. 6)
rho_init = ???     # initial value for the rho factor (see Eq. 6)
rho_endt = ???     # final value for the rho factor (see Eq. 6)
N_init = ???     # initial value for the scale factor (see Eq. 6)
N_endt = ???     # final value for the scale factor (see Eq. 6)
limits = ((beta_init,beta_endt),(phi_init,phi_endt),(N_init,N_endt))

# data 
h_n, k_n, l_n, I = ???  # file with the Miller indexes and the respective intensities


In [ ]:
Res  =  []
Phi  =  []
Beta =  []
Nor =   []
Thick = []
DW_mult = []
Peak_num =  []


i000 = i_dire(h,k,l)           

mf_grid = np.arange(mf_init, mf_endt + mf_step, mf_step)
t_grid = np.arange(t_init, t_endt + t_step, t_step)

for i in mf_grid:
    for j in t_grid:

       
        print('Initial:','Para_angle = ', beta0, 'Per_angle = ', phi0, 'Norm =', N0,'Thickness = ', j , 'DW_multiplier = ',i)
        print('Inital Residue = ', Residue([beta0,phi0,N0],pre,j,i,I,i000))
        
        
        # Optimization 
        # choose a proper minimization algorith
        fit=(minimize(Residue, bounds = limits, args = (pre, j, i ,I,i000)))
        print('Fit Residue =', fit.fun, '/  Fit Parameters =', fit.x )
        
        ## Results
        res = Residue(fit.x,pre,j,i,I,i000)

        
        Res.append(res)
        Beta.append((fit.x)[0])       
        Phi.append((fit.x)[1])
        Nor.append((fit.x)[2])
        Thick.append(j)
        DW_mult.append(i)
        Peak_num.append(len(I))
        print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Finding the right thickness and  mdb
print('XXXXX Results XXXXXXXXXX')
ind = np.argmin(Res)
res = Res[ind]
print('Final:','Para_angle = ', Beta[ind], 'Per_angle = ', Phi[ind], 'Norm =', Nor[ind],'Thickness = ', Thick[ind] , 'DW_multiplier = ',DW_mult[ind])
print('Final Residue = ', res)
        